In [40]:
from pyspark.sql import SparkSession

In [41]:
spark = SparkSession.builder.appName('nlp').getOrCreate()

In [42]:
from pyspark.ml.feature import Tokenizer, RegexTokenizer

In [43]:
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType

In [44]:
sen_df = spark.createDataFrame([
    (0,'Hi I heard about Spark'),
    (1,'I wish java could use case classes'),
    (2,'Logistic, regression, models, are, neat'),
], ['id', 'sentence'])

In [45]:
sen_df.show()

+---+--------------------+
| id|            sentence|
+---+--------------------+
|  0|Hi I heard about ...|
|  1|I wish java could...|
|  2|Logistic, regress...|
+---+--------------------+



In [46]:
tokenizer = Tokenizer(inputCol='sentence', outputCol='words')

In [47]:
regextokenizer = RegexTokenizer(inputCol='sentence', outputCol='words', 
                                pattern='\\W')

In [48]:
count_tokens = udf(lambda words:len(words), IntegerType())

In [49]:
tokenized = tokenizer.transform(sen_df)

In [50]:
tokenized.withColumn('tokens', count_tokens(col('words'))).show()

+---+--------------------+--------------------+------+
| id|            sentence|               words|tokens|
+---+--------------------+--------------------+------+
|  0|Hi I heard about ...|[hi, i, heard, ab...|     5|
|  1|I wish java could...|[i, wish, java, c...|     7|
|  2|Logistic, regress...|[logistic,, regre...|     5|
+---+--------------------+--------------------+------+



In [51]:
rg_tokenized = regextokenizer.transform(sen_df)

In [52]:
rg_tokenized.withColumn('tokens', count_tokens(col('words'))).show()

+---+--------------------+--------------------+------+
| id|            sentence|               words|tokens|
+---+--------------------+--------------------+------+
|  0|Hi I heard about ...|[hi, i, heard, ab...|     5|
|  1|I wish java could...|[i, wish, java, c...|     7|
|  2|Logistic, regress...|[logistic, regres...|     5|
+---+--------------------+--------------------+------+



In [53]:
from pyspark.ml.feature import StopWordsRemover

In [54]:
sentenceDataFrame = spark.createDataFrame([
    (0, ['I', 'saw', 'the', 'green', 'horse']),
    (1, ['Marry', 'had', 'a', 'little', 'lamb'])], ['id', 'tokens'])


In [55]:
remover = StopWordsRemover(inputCol='tokens', outputCol='filtered')

In [56]:
remover.transform(sentenceDataFrame).show()

+---+--------------------+--------------------+
| id|              tokens|            filtered|
+---+--------------------+--------------------+
|  0|[I, saw, the, gre...| [saw, green, horse]|
|  1|[Marry, had, a, l...|[Marry, little, l...|
+---+--------------------+--------------------+



In [57]:
#n-gram

In [58]:
from pyspark.ml.feature import NGram


In [59]:
wordDataFrame = spark.createDataFrame([
    (0, ["Hi", "I", "heard", "about", "Spark"]),
    (1, ["I", "wish", "Java", "could", "use", "case", "classes"]),
    (2, ["Logistic", "regression", "models", "are", "neat"])], ["id", "words"])


In [60]:
ngram = NGram(n=2, inputCol='words', outputCol='grams')

In [62]:
ngram.transform(wordDataFrame).select('grams').show(truncate=False)

+------------------------------------------------------------------+
|grams                                                             |
+------------------------------------------------------------------+
|[Hi I, I heard, heard about, about Spark]                         |
|[I wish, wish Java, Java could, could use, use case, case classes]|
|[Logistic regression, regression models, models are, are neat]    |
+------------------------------------------------------------------+

